In [71]:
import pandas as pd
import numpy as np
import nltk
import re
from pprint import pprint
import spacy
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


import spacy  ## spacy is their for lemmatization

# Plotting tools
import pyLDAvis
#from pyLDAvis import gensim
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline



# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import warnings
warnings.filterwarnings("ignore")

In [72]:
train=pd.read_csv('Usecase3_Dataset.csv')

In [76]:
Virgin_America=train[train.airline=="Virgin America"]["text"].values
United = train[train.airline=="United"]["text"].values
Southwest = train[train.airline=="Southwest"]["text"].values
Delta= train[train.airline=="Delta"]["text"].values
American= train[train.airline=="American"]["text"].values
US_Airways= train[train.airline=="US Airways"]["text"].values

In [79]:
data=train[train.airline=="Virgin America"]["text"].values

In [81]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = data.tolist()
data_words = list(sent_to_words(data))
#print(data_words[:1])

In [82]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take'])

In [83]:
import spacy
from spacy.cli.download import download
download(model="en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [84]:
# Building the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=1, threshold=300) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  

In [85]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

pprint(lda_model.print_topics())

[(0,
  '0.029*"love" + 0.020*"come" + 0.018*"amp" + 0.018*"day" + 0.017*"customer" '
  '+ 0.014*"contact" + 0.014*"email" + 0.014*"site" + 0.013*"status" + '
  '0.011*"work"'),
 (1,
  '0.073*"flight" + 0.036*"fly" + 0.024*"guy" + 0.020*"help" + 0.020*"book" + '
  '0.016*"long" + 0.016*"lax" + 0.016*"seat" + 0.014*"wish" + 0.014*"point"'),
 (2,
  '0.027*"way" + 0.024*"fare" + 0.022*"dm" + 0.021*"change" + 0.020*"airline" '
  '+ 0.016*"fee" + 0.016*"asap" + 0.015*"able" + 0.014*"add" + 0.014*"time"'),
 (3,
  '0.079*"co" + 0.048*"virgin" + 0.031*"still" + 0.027*"look" + 0.024*"sfo" + '
  '0.022*"plane" + 0.018*"awesome" + 0.017*"austin" + 0.014*"http" + '
  '0.011*"restr"')]


## Virgin America Airline Important topics ##

In [86]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.216218  0.031823       1        1  32.221607
3     -0.096759  0.116411       2        1  23.601519
0     -0.078128  0.047387       3        1  22.392390
2     -0.041331 -0.195621       4        1  21.784484, topic_info=        Term       Freq      Total Category  logprob  loglift
62        co  63.000000  63.000000  Default  30.0000  30.0000
20    flight  79.000000  79.000000  Default  29.0000  29.0000
81    virgin  38.000000  38.000000  Default  28.0000  28.0000
21       fly  39.000000  39.000000  Default  27.0000  27.0000
71     still  25.000000  25.000000  Default  26.0000  26.0000
...      ...        ...        ...      ...      ...      ...
589   silver   5.561820   6.987307   Topic4  -4.8804   1.2958
59        st   4.237320   5.177492   Topic4  -5.1524   1.3236
387      bin   4.284794   5.598951   Topic4  -5.1412   1.2565
1323  byackc   4.269082  12.035703   Topic4  -5.1449   0.4875
1324      nl   4.269082  12.035703   Topic4  -5.1449   0.4875

[176 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
128       1  0.085462      able
128       4  0.940085      able
1320      1  0.485282  actually
1320      2  0.485282  actually
0         4  0.903647       add
...     ...       ...       ...
386       3  0.988607      work
1326      1  0.083086    xrdtov
1326      2  0.498517    xrdtov
1326      4  0.332345    xrdtov
183       1  0.943107      year

[181 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 3])

## United Airline Important topics ##

In [88]:
data=train[train.airline=="United"]["text"].values

In [89]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = data.tolist()
data_words = list(sent_to_words(data))
#print(data_words[:1])

In [90]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take'])

In [91]:
# Building the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=1, threshold=300) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  

In [92]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

pprint(lda_model.print_topics())

[(0,
  '0.039*"united" + 0.038*"seat" + 0.030*"fly" + 0.023*"ticket" + '
  '0.020*"airline" + 0.019*"day" + 0.011*"problem" + 0.011*"lose" + '
  '0.011*"new" + 0.010*"find"'),
 (1,
  '0.036*"gate" + 0.033*"http_co" + 0.030*"customer" + 0.023*"leave" + '
  '0.022*"amp" + 0.021*"iah" + 0.020*"people" + 0.015*"change" + 0.015*"sit" + '
  '0.012*"agent"'),
 (2,
  '0.117*"flight" + 0.033*"bag" + 0.033*"hour" + 0.029*"delay" + 0.028*"plane" '
  '+ 0.027*"service" + 0.019*"still" + 0.019*"give" + 0.018*"tell" + '
  '0.015*"last"'),
 (3,
  '0.059*"wait" + 0.046*"time" + 0.019*"passenger" + 0.016*"call" + '
  '0.016*"travel" + 0.015*"miss" + 0.014*"almost" + 0.014*"airport" + '
  '0.013*"email" + 0.012*"home"')]


## United Airline Important topics ##

In [93]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.306050  0.058499       1        1  31.346449
3     -0.159763  0.251560       2        1  23.143229
0     -0.047506 -0.162684       3        1  22.783203
1     -0.098781 -0.147375       4        1  22.727119, topic_info=         Term         Freq        Total Category  logprob  loglift
6      flight  1023.000000  1023.000000  Default  30.0000  30.0000
257      wait   385.000000   385.000000  Default  29.0000  29.0000
256      time   300.000000   300.000000  Default  28.0000  28.0000
99     united   248.000000   248.000000  Default  27.0000  27.0000
90       seat   241.000000   241.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
406     break    34.239583    34.882751   Topic4  -5.2245   1.4630
565   amazing    34.017493    34.679859   Topic4  -5.2310   1.4623
0        care    32.186832    32.815624   Topic4  -5.2863   1.4623
759   confirm    33.935815    34.599275   Topic4  -5.2334   1.4622
1489     step    38.844831    39.750666   Topic4  -5.0983   1.4586

[155 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
534       3  0.976746  actually
50        4  0.993551     agent
94        3  0.991471   airline
42        2  0.987769   airport
167       2  0.997713    almost
...     ...       ...       ...
321       2  0.987739      well
668       3  0.995808      wifi
121       2  0.996951      work
650       1  0.994391      year
385       4  0.975120       yet

[127 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 1, 2])

## Southwest Airline Important Topics ##

In [97]:
data=train[train.airline=="Southwest"]["text"].values

In [98]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = data.tolist()
data_words = list(sent_to_words(data))
#print(data_words[:1])

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take'])

In [99]:
# Building the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=1, threshold=300) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

pprint(lda_model.print_topics())

## Southwest Important Topics ##

In [100]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.306050  0.058499       1        1  31.340422
3     -0.159763  0.251560       2        1  23.144433
0     -0.047506 -0.162684       3        1  22.795550
1     -0.098781 -0.147375       4        1  22.719595, topic_info=         Term         Freq        Total Category  logprob  loglift
6      flight  1023.000000  1023.000000  Default  30.0000  30.0000
257      wait   385.000000   385.000000  Default  29.0000  29.0000
256      time   300.000000   300.000000  Default  28.0000  28.0000
99     united   249.000000   249.000000  Default  27.0000  27.0000
90       seat   241.000000   241.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
406     break    34.228248    34.871502   Topic4  -5.2245   1.4633
565   amazing    34.006231    34.668690   Topic4  -5.2310   1.4626
0        care    32.176177    32.805053   Topic4  -5.2863   1.4626
759   confirm    33.924581    34.588137   Topic4  -5.2334   1.4626
1489     step    38.831972    39.737899   Topic4  -5.0983   1.4589

[155 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
534       3  0.976231  actually
50        4  0.993876     agent
94        3  0.998741   airline
42        2  0.987718   airport
167       2  0.997661    almost
...     ...       ...       ...
321       2  0.987689      well
668       3  0.995279      wifi
121       2  0.996899      work
650       1  0.994579      year
385       4  0.975435       yet

[127 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 1, 2])

In [102]:
data= train[train.airline=="Delta"]["text"].values

In [103]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = data.tolist()
data_words = list(sent_to_words(data))
#print(data_words[:1])

In [104]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take'])

In [105]:
# Building the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=1, threshold=300) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  

In [106]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

pprint(lda_model.print_topics())

[(0,
  '0.039*"wait" + 0.037*"great" + 0.022*"minute" + 0.021*"airport" + '
  '0.021*"today" + 0.020*"bag" + 0.018*"hour" + 0.016*"seat" + 0.015*"really" '
  '+ 0.013*"travel"'),
 (1,
  '0.032*"help" + 0.019*"pick" + 0.019*"check" + 0.016*"love" + '
  '0.014*"tomorrow" + 0.013*"call" + 0.013*"home" + 0.012*"look" + '
  '0.012*"book" + 0.011*"email"'),
 (2,
  '0.066*"co" + 0.044*"fly" + 0.028*"jetblue" + 0.021*"work" + 0.017*"well" + '
  '0.015*"passenger" + 0.014*"wall" + 0.014*"http" + 0.014*"airline" + '
  '0.011*"way"'),
 (3,
  '0.116*"flight" + 0.036*"delay" + 0.032*"time" + 0.025*"new" + 0.020*"amp" + '
  '0.019*"guy" + 0.016*"change" + 0.016*"follow" + 0.013*"plane" + '
  '0.013*"character"')]


## Delta Airline Important topics ##

In [107]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.249108 -0.102551       1        1  30.764196
2      0.181997 -0.198925       2        1  24.350247
1      0.056660  0.162448       3        1  23.899599
0      0.010452  0.139027       4        1  20.985958, topic_info=        Term        Freq       Total Category  logprob  loglift
14    flight  507.000000  507.000000  Default  30.0000  30.0000
0         co  229.000000  229.000000  Default  29.0000  29.0000
10       fly  150.000000  150.000000  Default  28.0000  28.0000
210    delay  157.000000  157.000000  Default  27.0000  27.0000
431     wait  115.000000  115.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
113    issue   14.539346   15.178801   Topic4  -5.3202   1.5183
196      app   16.065992   16.779115   Topic4  -5.2204   1.5179
295     hear   18.106900   18.992882   Topic4  -5.1008   1.5135
3587   elite   27.250648   56.753024   Topic4  -4.6920   0.8277
3588  silver   27.250648   56.753024   Topic4  -4.6920   0.8277

[165 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
940       3  0.978487    agent
8         2  0.991084  airline
230       4  0.985789  airport
364       3  0.969046  already
112       4  0.990385   always
...     ...       ...      ...
117       2  0.954900  website
268       2  0.985056     well
195       4  0.960933     wifi
249       2  0.974818     wish
189       2  0.996035     work

[148 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 1])

In [109]:
data= train[train.airline=="American"]["text"].values


In [110]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = data.tolist()
data_words = list(sent_to_words(data))
#print(data_words[:1])

In [111]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take'])

In [112]:
# Building the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=1, threshold=300) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  

In [117]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

pprint(lda_model.print_topics())

[(0,
  '0.038*"tomorrow" + 0.026*"time" + 0.025*"weather" + 0.024*"guy" + '
  '0.022*"ticket" + 0.019*"airport" + 0.019*"work" + 0.018*"dfw" + 0.017*"jfk" '
  '+ 0.016*"put"'),
 (1,
  '0.058*"help" + 0.038*"customer" + 0.035*"amp" + 0.028*"service" + '
  '0.020*"leave" + 0.019*"tell" + 0.019*"bag" + 0.017*"bad" + 0.014*"ever" + '
  '0.014*"response"'),
 (2,
  '0.190*"flight" + 0.062*"cancel" + 0.041*"hour" + 0.022*"http_co" + '
  '0.020*"delay" + 0.019*"seat" + 0.016*"late" + 0.015*"flightle" + '
  '0.013*"today" + 0.013*"book"'),
 (3,
  '0.064*"call" + 0.043*"aa" + 0.031*"hold" + 0.029*"wait" + 0.027*"phone" + '
  '0.026*"agent" + 0.021*"plane" + 0.019*"minute" + 0.015*"back" + '
  '0.015*"still"')]


## American Airline Important Topics ##

In [118]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.202650  0.224304       1        1  27.558489
2      0.287256  0.108830       2        1  25.106219
1     -0.040826 -0.132453       3        1  24.355356
0     -0.043780 -0.200681       4        1  22.979936, topic_info=        Term        Freq       Total Category  logprob  loglift
14    flight  931.000000  931.000000  Default  30.0000  30.0000
77      call  343.000000  343.000000  Default  29.0000  29.0000
42    cancel  303.000000  303.000000  Default  28.0000  28.0000
50      help  274.000000  274.000000  Default  27.0000  27.0000
20        aa  230.000000  230.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
16     point   29.332403   30.029404   Topic4  -5.0313   1.4471
85       fix   29.528081   30.229878   Topic4  -5.0247   1.4471
869      car   27.751144   28.424819   Topic4  -5.0868   1.4466
1097     buy   30.594406   31.408834   Topic4  -4.9892   1.4443
646     full   33.271617   35.186833   Topic4  -4.9053   1.4146

[157 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
20        1  0.996489         aa
169       1  0.992732      agent
9         1  0.969965    airline
59        4  0.988112    airport
358       3  0.988556   american
...     ...       ...        ...
392       4  0.990751    weather
273       3  0.971346       well
281       4  0.988354       work
1026      2  0.961660      wrong
215       4  0.988272  yesterday

[129 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 1])

In [120]:
data= train[train.airline=="US Airways"]["text"].values

In [121]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = data.tolist()
data_words = list(sent_to_words(data))
#print(data_words[:1])

In [122]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take'])

In [123]:
# Building the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=1, threshold=300) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  

In [124]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

pprint(lda_model.print_topics())

[(0,
  '0.039*"reservation" + 0.034*"delay" + 0.033*"phone" + 0.024*"tell" + '
  '0.023*"miss" + 0.018*"http_co" + 0.017*"hope" + 0.016*"clt" + 0.015*"phl" + '
  '0.014*"work"'),
 (1,
  '0.031*"bag" + 0.030*"charlotte" + 0.020*"pay" + 0.019*"give" + 0.018*"look" '
  '+ 0.015*"contact" + 0.015*"great" + 0.015*"home" + 0.014*"sit" + '
  '0.012*"vacation"'),
 (2,
  '0.037*"today" + 0.034*"help" + 0.033*"day" + 0.025*"amp" + 0.025*"back" + '
  '0.022*"service" + 0.021*"customer" + 0.020*"number" + 0.019*"airline" + '
  '0.016*"email"'),
 (3,
  '0.146*"flight" + 0.038*"hour" + 0.037*"hold" + 0.025*"time" + 0.021*"call" '
  '+ 0.020*"gate" + 0.020*"wait" + 0.020*"plane" + 0.019*"change" + '
  '0.018*"still"')]


## US Airways Airline Important Topics

In [125]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.311223 -0.014851       1        1  33.155142
0     -0.119122 -0.266065       2        1  24.794497
2     -0.106148  0.144932       3        1  22.696828
1     -0.085952  0.135983       4        1  19.353533, topic_info=             Term         Freq        Total Category  logprob  loglift
2          flight  1022.000000  1022.000000  Default  30.0000  30.0000
25           hour   267.000000   267.000000  Default  29.0000  29.0000
117   reservation   207.000000   207.000000  Default  28.0000  28.0000
4            hold   258.000000   258.000000  Default  27.0000  27.0000
40          today   176.000000   176.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
587         money    23.384620    24.128851   Topic4  -5.1663   1.6110
395       amazing    23.643429    24.443656   Topic4  -5.1553   1.6090
352     different    22.705504    23.495299   Topic4  -5.1958   1.6081
672         month    26.355712    27.424284   Topic4  -5.0467   1.6026
1059        carry    39.728985    43.240633   Topic4  -4.6363   1.5576

[159 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
697       2  0.969596       aa
8         1  0.983278    agent
362       3  0.990031  airline
78        2  0.992417  airport
186       3  0.984032   airway
...     ...       ...      ...
28        1  0.987586      way
211       2  0.988382  weather
642       2  0.983760  website
6         1  0.993571     well
268       2  0.995449     work

[136 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2])